Last modified (yyyy/mm/dd): 2024/08/08
Author: Alessandro Ulivi (ale.ulivi@gmail.com)

The following structure is expexted.
- input_folder.
    - sample_1.
        - arx2_binary_timecourse.
        - cyk1_binary_timecourse.
        - pip2_binary_timecourse.
        - embryo_binary_timecourse.
    ...
    - sample_n.
        - arx2_binary_timecourse.
        - cyk1_binary_timecourse.
        - pip2_binary_timecourse.
        - embryo_binary_timecourse.

- output_folder. It can contain anything. The pipeline will automatically generate sub-folders inside output_folder. One sub-folder per "sample_folder" which is inside "input_folder" will be generated. Insided the sub-folder there will be the quantification (.csv file) for the sample.

- roi_folder.
    - roi_exclude.
        - sample_1
            - .roi / .zip file.
        - sample_n
            - .roi / .zip file.
    - roi_maintain.
        - sample_1
            - .roi / .zip file.
        - sample_n
            - .roi / .zip file.

NOTE:
- input_folder, ouput_folder, roi_folder can have any name. However roi_exclude and roi_maintain names CANNOT be changed.
- Both roi_exclude and roi_maintain folders can be empty or alternatively they can contain sub-folders. If they contain sub-folders, the names of the sub-folders must match the names of the folders (the samples) which are inside input_folder as the name is used to assign the roi to the sample. Only sub-folders for the samples in input_folder which must undergo the exclusion/maintenance of a part of the image must be created. It is not required that per each folder in roi_exclude a corresponding folder in roi_maintain is present.
- in each "sample_folder" of both roi_exclude and roi_maintain a single .roi/.zip file is expected. Note that if multiple ROIs are defined they can be included in the .zip file.

IMPORTANT NOTE: about the use of ROIs. It is possible to use one of the files within a sample_folder as an roi_file. It must have the same shape of the other file. It will not be included in the measurements quantification. It will effectively define the portion of files where the analysis is done. For example, if one has a segmentation for an embryo, the cortex of an embryo, a cell etc, it is possible to use it as an ROI. It is not required that the file is binary, but it will be binarized in the process (refer to quantify_channels within multi_channels_multi_quantifications.py). If one of the files within a sample_folder has to be used as an roi_file, indicate how to unequivocally identify it in the variable ch_roi below.
It is further possible to change the ROI-specific analysis by using files included in the sub-folders of the roi_exclude and roi_maintain folders. It is a redundant action, but it allows for complex analyses as, for example, limiting the quantification to specific parts of the files or avoiding the quantification of a structure only along an axis and at specific timepoints.

The following cell imports required packages and functions. It also initializes variables which are important for the analysis to run

The following cell must be run.

Don't modify the following cell.

In [1]:
import os
import numpy as np
import random
import tifffile
from utils import listdirNHF
import matplotlib.pyplot as plt
from apply_quantification import sample_quantifier

#Initialize analysis axis as None - Don't modify the variable, it will be overwritten, if necessary, later
analysis_axis = None

#Initialize roi positioning axis as None - Don't modify the variable, it will be overwritten, if necessary, later
roi_positioning_axis = None

#Initialize roi_3D_maintain as False - Don't modify the variable, it will be overwritten, if necessary, later
roi_3D_maintain = False

#Initialize roi_3D_exclude as False - Don't modify the variable, it will be overwritten, if necessary, later
roi_3D_exclude = False

#Initialize roi_positioning_axis as None - Don't modify the variable, it will be overwritten, if necessary, later
roi_positioning_axis = None


The following cell defines the input_folder, output_folder and the roi_folder. It also defines the names of the analyzed channels. The function which does the measurements is generic, runs on any image and saves the results using a generic name. The cell below allows to indicate the specific name which will identify the measurements per each file in "sample sub-folder" of input_folder.

The following cell must be run.

The following cell should be modified.

In [2]:
#Indicate the input folder. Indicate the full path of the input folder. Refer to the initial description for the structure underlying the input folder.
input_folder = r"/Users/ulivia/Desktop/Alessandro/projects/filopodia_paper/fig1_actin_nucleators/e_cyk1_arx2_pip2/step_04/240620_AU_ACR074"

#Indicate the output folder. Indicate the full path of the output folder. Refer to the initial description for the structure underlying the output folder.
output_folder = r"/Users/ulivia/Desktop/Alessandro/projects/filopodia_paper/fig1_actin_nucleators/e_cyk1_arx2_pip2/step_05/240620_AU_ACR074"

#Indicate the roi folder. Indicate the full path of the roi folder. Refer to the initial description for the structure underlying the roi folder.
#NOTE: if no roi has to be used, it is possible to associate this variable to None (roi_folder = None)
roi_folder = r"/Users/ulivia/Desktop/Alessandro/projects/filopodia_paper/fig1_actin_nucleators/e_cyk1_arx2_pip2/step_04/240620_AU_ACR074_roi"

#=================================
#For certain measurements, it is possible to obtain the result also for imaged where the position of individual pixels is randomly shuffled, in order to obtain a comparison with random
#chances. Indicate here the number of shuffling times to use in the analysis. Indicate 0 if no shuffling is required.
#For the moment the only measurements which allow shuffling are measure_pixels_overlap in co_localization_measurements.py and analyse_spearman_3Darray in signal_correlation.py. Refer
#to their documentation
shuffle__times = 10 #Int

#=================================
#=================================
#=================================
#Identify channels - Indicate here the name through which individual files within input_folder will be identified in the dataframe containing the measurement results
#For example, if file ch_0 is indicated as "arx2", there will be the column "arx2_measurementX" for the measurementX in the dataframe containing the measurement.
#(For the moment) one variable must be created per each file in a sample_folder. NOTE 1: the identification name MUST be a sub-string (a sub part) of the file name.
# NOTE 2: the use of the names ch_0, ch_1, ch_2 for the variables is completely arbitrary.
ch_0 = 'arx2' #string
ch_1 = 'cyk1' #string
ch_2 = 'pip2' #string

#Identify the roi_file - if one of the files within the sample_folder should be used as ROI and not for quantification, indicate how to identify it unequivocally from the other files,
#by reporting a part of its name, as a string. If no file within the sample_folder should be used as ROI, indicate this variable as None.
ch_roi = 'embryo' #None or string


The following 2 cells are relevant only if files should be analysed along an axis.
Run and modify the following 2 cells only if files should be analysed along an axis.
Read the following description only if files should be analysed along an axis.

It is possible to analyse files along a dimension (aka axis). For example, if files are timecourses, it is possible to obtain the quantifications for each individual timepoint. In order to do this it is required to specify the axis of the file to be analysed.
The following cell prints the shape of the input files to allow the user to identify, if required, an analysis axis. The ouput of the cell looks like this (axis0, axis1,axis2,axis3 etc etc). There are as many axes as the dimensions of the file. A 2D image has 2 axes. A stack and a timecourse of single planes have 3 axes, and so on.
Note that the number of the axis name starts from 0 in python. I named it like this to match the axis position, which, in python, starts from 0 and goes from left to right. So axis0 is in position 0, axis1 is in position 1 and so on.
The actual number which is displayed will be, however, the size of the axis. For example, if a stack is made of 10 z planes and each plane is a 1024*1024 pixels image, the output (the shape of the file) will be one of the following 3 options: (24,1024,1024), (1024,24,1024), (1024,1024,24).
If the analysis should be performed per each plane of the z axis, one must indicate the position of the 24 (aka the position of the z dimension) in the variable analysis_axis (in 2 cells). Remember that the position counting starts from 0 and goes from left to right. So if for example the output is (24,1024,1024), to analyse the z axis set analysis_axis=0 (in 2 cells). If the output is (1024,24,1024), to analyse the z axis set analysis_axis=1 (in 2 cells). Finally, if the output is (1024,1024,24), to analyse the z axis set analysis_axis=2 (in 2 cells).

In [8]:
#PRINT THE SHAPE OF A FILE TO UNDERSTAND THE AXIS TO ANALYSE, IF REQUIRED - DON'T MODIFY THIS CELL

#Open a random file

#Get a random folder in input_folder
random_input_folder = random.choice(listdirNHF(input_folder))

#Form the directory of the random folder
random_input_folder_dir = os.path.join(input_folder, random_input_folder)

#Get a random file in the random input folder
random_file = random.choice(listdirNHF(random_input_folder_dir))

#For the file full path
random_file_full_path = os.path.join(random_input_folder_dir, random_file)

#Open the file
random_file = tifffile.imread(random_file_full_path)

#Print the shape of the file
print("this is the shape of the input files: ", random_file.shape)


this is the shape of the input files:  (61, 928, 684)


In [3]:
#INDICATE THE POSITION OF THE ANALYSIS AXIS, IF REQUIRED
analysis_axis=0 #Substitute the number with the position of the analysis axis - the position must be a number without decimals

#Optional - it is possible to specify a name for the column where the position along the analysis_axis is stored in the quantification dataframe
#For example, if the analysed arrays are timepoints of a timecourse, it is possible to call it "timepoint".
#If the variable is left as none the column will be called "axis_"+(analysis_axis+1) NOTE: the +1 derives from the fact that an extra dimension is added in the analysis
name_iteration_axis = 'timepoint' #None or string

#Don't modify the following lines
#Transform analysis_axis to int if a float value is indicate
if not isinstance(analysis_axis, int):
    analysis_axis = int(analysis_axis)

The following 2 cells are relevant only if ROIs to be excluded and ROIs to be maintained are defined for certain samples.

Run the cells only if ROIs to be excluded and ROIs to be maintained are defined for certain samples.

It could be useful to anyway read the following explanation.



It is possible to analyse specific regions of interest (ROIs) in the files. These can be defined by including a roi_mask file in the sample_folder and specify it in the ch_roi variable above. In parallel and in addition, it can be done by using .roi/.zip files in the sub-folders of roi_exclude and roi_maintain.

When all the three methods are used, and some pixels are in common between the three ROIs, the priority goes:
- roi to exclude.
- (roi to maintain) AND (roi file in the sample_folder).

This means that if pixels are indicated to be excluded, they will be excluded irrespective of the other two ROIs. If no exclusion criteria is given, the pixels will be analysed only if they are within the ROI of both roi to maintain and the file in sample_folder.


I will further describe here the use of roi_to_maintain and roi_to_exclude, thus the following lines don't refer to ROIs defined in the file within the sample_folder.

It is assumed that ROIs are generated on one of the files to analyse. Also, to the whole process was conceptualized for generating ROIs using Fiji/ImageJ.

I will use roi_maintain for the explanation, but the exact procedure applies to roi_exclude.

- If no ROI should be used. Don't include the folder with the sample name in roi_maintain folder. When no sample at all needs a ROI, it is possible to indicate the variable roi_folder above as None, and don't even create a roi_folder.

- If the files to analyse are 2D. The variable roi_3D_maintain must be false (roi_3D_maintain=False). In this case the value of roi_positioning_axis is irrelevant. Roi files can be .roi (a single ROI) or .zip (multiple ROIs within the same 2D image)

- If the files to analyse are 3D and the ROIs are applied identically to all the 2D frames of the "3rd dimension" (for example, a z-stack or timepoints of a timecourse). There are 2 possibilities.
1) The first possibility is to generate all the ROIs on a single 2D frame and save them in a file (.roi if a single ROI or .zip if multiple ROIs). Indicate roi_3D_maintain as False (roi_3D_maintain=False). Specify the "3rd dimension" along which to propagate the ROIs. If the "3rd dimension" is identical to the analysi_axis, the variable roi_position_axis can remain None. The analysis_axis will be automatically interpreted as the axis along which to propagate the ROIs (NOTE: an error will raise if no analysis axis is provided and roi_position_axis=None). If the "3rd dimension" is different than the analysis_axis or the files don't have to be analysed along an axis, indicate it in the variable roi_position_axis. For example, the file to analyse is a 3D z-stack of 10 planes and each plane is a 1024*1024 pixels image. The shape is (10,1024,1024). The same ROI should be used on all the planes. The roi is drawn on the first z-plane and saved as a .roi file. I indicate roi_3D_maintain=False. I indicate roi_position_axis=0 (the position of the z dimension in the shape). If the file had to be analysed along the z-dimension, independently per each plane, because analysis_axis was already defined, I could have left roi_position_axis=None.
2) The second possibility is to generate the ROIs, identical, in each 2D frame of the "3rd dimension" of the files to analyse and save the file as a .zip. Indicate roi_3D_maintain as True (roi_3D_maintain=True). Specify the "3rd dimension" along which to position the ROIs. If the "3rd dimension" is identical to the analysi_axis, the variable roi_position_axis can remain None. The analysis_axis will be automatically interpreted as the axis along which to position the ROIs (NOTE: an error will raise if no analysis axis is provided and roi_position_axis=None). If the "3rd dimension" is different than the analysis_axis or the files don't have to be analysed along an axis, indicate it in the variable roi_position_axis. For example, the file to analyse is a 3D z-stack of 15 planes and each plane is a 1024*1024 pixels image. The shape is (1024,10,1024). The same ROI should be used on all the planes. The roi is drawn, identical, on each z-plane and the collection of ROIs is saved as a .zip file. I indicate roi_3D_maintain=True. I indicate roi_position_axis=1 (the position of the z dimension in the shape). If the file had to be analysed along the z-dimension, independently per each plane, because analysis_axis was already defined, I could have left roi_position_axis=None.

- If the files to analyse are 3D and different ROIs are applied to different 2D frames along the "3rd dimension". Save the collection of ROIs as a .zip file. Indicate roi_3D_maintain as True (roi_3D_maintain=True). Specify the "3rd dimension". If the "3rd dimension" is identical to the analysi_axis, the variable roi_position_axis can remain None. The analysis_axis will be automatically interpreted as the axis along which to position the ROIs (NOTE: an error will raise if no analysis axis is provided and roi_position_axis=None). If the "3rd dimension" is different than the analysis_axis or the files don't have to be analysed along an axis, indicate it in the variable roi_position_axis.
Example 1. A multi-position file is analysed. There are 6 positions, each corresponding to a well of a multi-well. Per each position there is a 2D image of size 1024*1024 pixels. The shape of the file is (1024,1024,6) thus indipenent xy position are at position 2 of the file's shape. 1 round ROI (the well of the multi-well) is analysed at each position, but the shape of the ROI slightly changes at each position. The measurement is done globally, thus the results of the ROI are all pooled together and no analysis_axis is required. I open the file as a stack in ImageJ. I define an ROI at each 2D plane, and I save the collection of ROIs as a .zip file. I indicate roi_3D_maintain=True. I indicate roi_position_axis=2 (NOTE: because there is no analysis_axis, in this case one must specify a roi_position_axis).
Example 2. A timecourse is analysed. There are 20 timepoints. Each timepoint is a 512*512 2D image. The shape of the file is (20,512,512), thus the "time dimension" is in position 0 of the file's shape. I'm interested at a cell which moves, I am only interested at timepoints 10, 12 and 15. I want an independent measurement per each timepoint, for this reason I set analysis_axis=0. I draw ROIs at timepoints 10,12 and 15 and I save the collection as a .zip file. I indicate roi_3D_maintain as True (roi_3D_maintain=True). Because analysis_axis has been defined and it is the same as roi_position_axis I can leave roi_position_axis=None. NOTE: the analysis will be done on the whole image for all the timepoints except timepoints 10, 12 and 15, while it will be restricted to the respective ROIs for the 3 timepoints. One would then have to extract the interesting values from the results. This was just an example...

FINAL NOTE: roi_maintain and roi_exclude are independent but for 1 aspect. If roi_3D_maintain and roi_3D_exclude are both set to True, the axis along which to position the ROIs must be the same (there is only one roi_position_axis variable and one analysis_axis variable).


In [4]:
#PRINT THE SHAPE OF A FILE TO UNDERSTAND THE AXIS ALONG WHICH TO POSITION ROIs, IF REQUIRED - DON'T MODIFY THIS CELL
#NOTE: THIS CELL IS COMPLETELY REDUNDANT, IT IS, IN FACT, IDENTICAL TO TWO CELLS ABOVE.

#Open a random file

#Get a random folder in input_folder
random_input_folder = random.choice(listdirNHF(input_folder))

#Form the directory of the random folder
random_input_folder_dir = os.path.join(input_folder, random_input_folder)

#Get a random file in the random input folder
random_file = random.choice(listdirNHF(random_input_folder_dir))

#For the file full path
random_file_full_path = os.path.join(random_input_folder_dir, random_file)

#Open the file
random_file = tifffile.imread(random_file_full_path)

#Print the shape of the file
print("this is the shape of the input files: ", random_file.shape)

this is the shape of the input files:  (301, 928, 684)


In [4]:
#Indicate if files in the sub_folders of roi_maintain should interpreted as 3D files
roi_3D_maintain = False #False or True

#Indicate if files in the sub_folders of roi_maintain should interpreted as 3D files
roi_3D_exclude = False #False or True

#Indicate the axis to use for positioning roi files when the are 3D
roi_positioning_axis = None #None or int

#Don't modify the following lines
#Assert that if rois are 3D roi_positioning_axis and analysis_axis are not None at the same time
if roi_3D_maintain==True or roi_3D_exclude==True:
    if roi_positioning_axis==None:
        assert analysis_axis!=None, "if one or both roi_3D_maintain and roi_3D_exclude are True, roi_positioning_axis and analysis_axis can't be both None"

The following cell run the batch processing.

The following cell must be run.

The following cell should not be modified.

In [5]:
#Initialize a quantifier object
channels_quantifier = sample_quantifier(analysis_axis=analysis_axis,
                                        roi_structure=ch_roi,
                                        shuffle_times=shuffle__times,
                                        no_quantification_valu_e=np.nan,
                                        channels_binarization_thresholds=0,
                                        transform_to_label_img=True,
                                        get_mask_area_val_4zero_regionprops=0,
                                        count_regions_number_threshold_roi_mask=0,
                                        n_of_region_4areas_measure=0,
                                        reg_eucl_dist_within_arr_val_n_regions_nopass=1,
                                        get_convex_hull_min_px_num=2,
                                        min_px_over_thresh_common=-1,
                                        measure_pixels_overlap_n_px_thr_1=1,
                                        measure_pixels_overlap_n_px_thr_2=0,
                                        count_n_overl_reg_intersection_threshold=None,
                                        conv_hull_fract_px_thre_arr_1=3,
                                        conv_hull_fract_px_thre_arr_2=3,
                                        get_conv_hull_fract_arr1_NOpass_arr2_pass_v=0.0,
                                        get_conv_hull_fract_arr2_NOpass_v=np.nan)

#Start batch processing
channels_quantifier.batch_quantification(root_input_folder=input_folder,
                                         root_output_folder=output_folder,
                                         channels_new__names=[ch_0, ch_1, ch_2], #MODIFY THIS DEPENDING ON THE VARIABLES DEFINED IN CELL 5 (including markdowns)
                                         root_folder_roi=roi_folder,
                                         new_name_iteration__axis=name_iteration_axis,
                                         roi_3D__maintain=roi_3D_maintain,
                                         roi_3D__exclude=roi_3D_exclude,
                                         roi_position__axis=roi_positioning_axis)



--- 240620_AU_ACR074_07_03
(3, 61, 928, 684)
(61, 928, 684)
--- 240620_AU_ACR074_09_03
(3, 61, 928, 684)
(61, 928, 684)
FINISHED!
